## Line 1 Testing 

#### Graph Structure Tests
- Finite vertices — needs testing (no explicit finiteness validation)

- Supports directed and bidirected edges — already tested (graph construction and conversions in test_graph.py)
- Directed edges connect existing vertices — implicitly covered by construction; explicit test would be better
- Bidirected edges connect existing vertices — implicitly covered by construction; explicit test would be better
- Algorithm detects cycles — already tested (test_graph.py:test_is_acyclic)
- Cycle detection returns True for a known cycle (e.g., v1→v2→v3→v1) — already tested (cyclic example present)
- Cycle detection returns False for acyclic graphs — needs testing (add explicit acyclic case)
- No directed edge goes from a vertex to itself — needs testing (no explicit self-loop rejection)
- No bidirected edge goes from a vertex to itself — needs testing (no explicit self-loop rejection)
- Graph construction rejects attempts to add self-loops (directed and bidirected) — needs testing
- Bidirected edges are symmetric (if vi↔vj exists, then vj↔vi must exist) — partially implicit via deduplication (test_is_not_multigraph); explicit symmetry guarantee still needs testing
- Adding a bidirected edge automatically creates its symmetric pair — needs testing (auto-creation vs normalization not explicitly asserted)
- Directed edges allow cycles through intermediates (v1→v2→v3→v1 allowed) — already tested (cycle present in test_is_acyclic)
- Mixed edges are allowed (both vi→vj and vi↔vj can exist simultaneously) — needs testing (explicit coexistence test)
--- 

#### Variable Set Tests (11 tests) — all need testing
- Test that all variable sets (V, Y, W, J) are finite
- Test that each variable has a unique identifier within a set
- Test that variable sets are frozenset type (or the chosen immutable type)
- Test that frozensets cannot be modified after creation (immutability)
- Test that frozensets are hashable (can be used as dict keys or in other sets)
- Test that sets are unordered (ordering doesn't affect equality)
- Test that algorithm results are independent of input ordering (permutation invariance)
- Test that Y and W are disjoint (Y ∩ W = ∅ is enforced)
- Test that overlapping Y and W causes the algorithm to fail at Line 2
- Test that J can overlap with Y (J ∩ Y ≠ ∅ is allowed)
- Test that J can overlap with W (J ∩ W ≠ ∅ is allowed)

---

#### Distribution Tests (8 tests) — all need testing

- Test that all probability values are in valid range [0, 1]
- Test that distribution is normalized (sum/integral equals 1 within tolerance)
- Test that distribution with negative values fails validation
- Test that all realizable configurations have positive probability (positivity)
- Test that distributions with structural zeros are flagged as potential issues (if required)
- Test that distribution respects independencies encoded in graph (Markov compatibility via σ-separation) using an actual distribution
- Test that distribution generated from a known graph satisfies the Markov property
- Test that post-intervention distribution P(V|do(J)) is consistent with mutilated graph G_{\overline{J}} (intervention semantics)
---

#### Intervention Operator Tests (8 tests)
- Test that do(X) removes all incoming directed edges to X (graph surgery) — already tested (test_graph.py:test_intervention)
- Test that edges not involving X as target remain unchanged — already tested (test_graph.py:test_intervention)
- Test that bidirected edges (confounders) remain after intervention — needs testing and clarify semantics: current expected graph in test_intervention suggests undirected edges incident to X are removed; if you intend they should remain, update implementation/tests accordingly
- Test that intervention operator is idempotent (do(X, X) ≡ do(X)) — needs testing
- Test that intervention operator is commutative for disjoint sets (do(X, Y) ≡ do(Y, X)) — needs testing
- Test that applying interventions in different orders gives the same graph — needs testing
- Test that empty intervention is identity (do(∅) returns original graph unchanged) — already tested (test_graph.py:test_intervention)
- Test that empty intervention preserves distribution (P(Y|do(∅)) = P(Y)) — needs testing

## Line 2 Testing (Precondition testing)

- Y ⊄ V → immediate FAIL.
- W ⊄ V → immediate FAIL.
- Y ∩ W ≠ ∅ → immediate FAIL.
- Valid case passes: Y ⊆ V, W ⊆ V, Y ∩ W = ∅ proceeds.
- Input ordering invariance: permutations of Y/W inputs don’t change behavior.

## Line 3 Testing - Ancestral Closure

- H changes after removing W: construct graph where removal of W alters ancestor set; verify H computed correctly.
- Invariants: Y ⊆ H and H ⊆ V \ W; H ∩ W = ∅.
- Ancestral closure idempotence: Anc^{G_{V\W}}(H) = H.
- Bidirected edges excluded from directed ancestor paths: confounder-only links don’t affect H.

## Line 4 Testing - Loop over Consolidated Districts 

- CD(H) partitions H: union equals H and districts are pairwise disjoint.
- Acyclic/no-confounder case: districts reduce to singletons.
- Cycles/confounders case: multi-node consolidated district appears.
- Edge case: H itself is a single district (|CD(H)| = 1).

## Line 5 Testing - IDCD Invocation (recursive algorithm)

- Base case: A = C returns Q[A].
- Failure case: A = D returns FAIL.
- Recursive case: C ⊂ A ⊂ D triggers decomposition and recursion on a minimal example.
- Cd^G(C) = C for district C from CD(H) (trivial consolidation).
- Q keys are hashable/retrievable (set-based indexing works).


## Line 6-9 Testing - Failure Handling and Loop Completion

- Early exit: first failing district causes immediate return FAIL (later districts not processed).
- Mid-loop failure: failing at a later district still halts immediately.
- All-success path: no FAIL values → loop completes and proceeds to product.


## Line 10 - Product Reconstructuion 

- Explicit product test: combine two independent district factors; Q[H] equals manual product.
- Commutativity: reordering districts yields identical Q[H].
- Associativity: different groupings yield identical Q[H].
- Singleton district: Q[H] == Q[C] when |CD(H)| = 1.

## Line 11 - Marginalization

- Identity marginalization: H = Y → integral over empty set returns Q[H] unchanged.
- Simple marginal example: H = {A,B}, Y = {B}; integral over A matches expected P(B|do(...)) (symbolic or tiny numeric example).
- Normalization after marginalization: result sums/integrates to 1 over Y’s support.


## Line 12 - Ending the function

- Return type/interface

    - Assert the function returns either a valid causal estimand (expression/distribution object) or the FAIL sentinel.
    -  Verify the returned object supports expected methods (e.g., evaluation, serialization) if applicable.

## Line 13 - Function IDCD 

#### IDCD Function Invocation and Validation

- Test that IDCD can be called with (G, C, D, Q[D]) and returns either a distribution over a set or FAIL.
- Test that return type is consistent (e.g., a distribution/factor-like object or a defined FAIL sentinel).

--- 

#### Input Type and Shape Checks

- Test G is a valid NxMixedGraph; passing a wrong type raises or returns FAIL.
- Test C and D are immutable sets of Variable (e.g., frozenset[Variable]); wrong types trigger FAIL.
- Test Q[D] is a distribution/factor defined exactly over D (its variables match D); mismatch triggers FAIL.
- Test Q[D] has a valid probability table (shape, domain) for D; malformed tables trigger FAIL.
---

#### Preconditions from Line 14 (enforced at entry to IDCD)

- Test C ⊆ D: if C ⊄ D, then FAIL.
- Test D ⊆ V (graph nodes): if D contains a variable not in G, then FAIL.
- Test CD(G_D) = {D}: in the induced subgraph G[D], D must be a single consolidated district; if not, FAIL.
    - Positive case: a small example where D is one consolidated district.
    - Negative case: D splits into multiple districts in G[D] → FAIL.
---

#### No Mutation and Determinism

- Test inputs not mutated: G, C, D, and Q[D] remain unchanged after calling IDCD.
- Test determinism: repeated calls with the same (G, C, D, Q[D]) produce identical outputs.
---

## Line 14 - Preconditions for IDCD 

#### Precondition Enforcement (Positive Cases)

- Valid C ⊆ D: construct small graph G with V={A,B,C}, choose D={B,C}, C={C}; assert IDCD proceeds (does not return FAIL).
- Valid D ⊆ V: all variables in D exist in G; assert non-FAIL.
- Valid consolidated district: in the induced subgraph G[D], D forms exactly one consolidated district; assert non-FAIL.
---

#### Precondition Enforcement (Negative Cases)

- C ⊄ D → FAIL: choose C={A}, D={B,C}; assert IDCD returns FAIL.
- D ⊄ V → FAIL: include a variable X∉V in D (e.g., D={B,X}); assert FAIL.
- CD(G_D) ≠ {D} → FAIL: construct G where the induced subgraph on D splits into ≥2 districts (e.g., no bidirected edges linking components), assert FAIL.

---
#### Consolidated District Specifics

- Positive CD(G_D) = {D} (single district): build D with bidirected edges or strongly connected directed components such that consolidated district unites all of D.
- Negative CD(G_D) ≠ {D} (multiple districts): remove bidirected edges and directed SCC links so D decomposes; verify FAIL.

---
#### Type and Shape Checks (Tied to Precondition Logic)

- C and D type: must be immutable sets (e.g., frozenset[Variable]); pass a list or mutable set triggers FAIL (or TypeError), depending on your design.
- Q[D] coverage: ensure Q[D] is defined exactly over D; mismatch (missing variable or extra variable) triggers FAIL.

---
#### No Mutation and Determinism

- Inputs not mutated: G, C, D, and Q[D] unchanged after IDCD returns (even on FAIL).
- Determinism: repeated runs with same (G, C, D, Q[D]) yield identical result (either consistent distribution object or FAIL).